# Figure 7 - Layout

In [ ]:
import sys
sys.path.append("..")

from main import *
from zebrafish import *
from visualization import *

from scipy.stats import zscore

# Loading figure data

In [ ]:
atlas_path = '/home/anleg84/Documents/Atlas/Mapzebrain/'
atlas = Mapzebrain(atlas_path)

neurons = Neurons(atlas)

somas = atlas.trim_centroids(neurons.somas)
somas = np.round(somas).astype('int')

mask_tectum = atlas.get_region_mask(22)
mask_neuropil = atlas.get_region_mask(17)

neurons_in_tectum = np.where(mask_tectum[somas[:, 2], somas[:, 1], somas[:, 0]] > 0)[0]


diagonal_values = np.load('../Results/figure7_diagonal_correlations.npy')
piecewise_fit = np.load('../Results/figure7_piecewise_fit.npy')
x = np.arange(50)

predicted_radius = np.load('../Results/figure7_predicted_radius.npy')
radius_lower, radius_upper = np.load('../Results/figure7_predicted_radius_interval.npy')
radii1 = np.load('../Results/figure7_connectivity_radii_method1.npy')
radii2 = np.load('../Results/figure7_connectivity_radii_method2.npy')

# Inspecting single neurons

Manually picking nice looking neurons

In [ ]:
selected_ids = [35, 78, 109, 126, 148, 273, 298, 336, 342, 346, 543, 605, 706, 753, 810, 895, 913, 1001, 1016, 1043, 1088, 3284, 3324, 3496, 3617, 3618, 3815, 3830, 4177, 4212, 4216, 4246, 4253, 4277, 4289]

for i in selected_ids:

    fig, ax = plt.subplots(figsize=(5, 5))
    color = plt.get_cmap('rainbow')(0)
    ax.imshow(atlas.XYprojection, cmap='binary', alpha=0.25)
    neurons.plot_top(ax, i, color=color, alpha=0.75, soma_size=2, linewidth=0.25, terminals=True, terminal_size=0.25)
    ax.set_xlim([280, 470])
    ax.set_ylim([500, 250])
    plt.title(i)
    plt.show()

# Making figure layout

In [ ]:
def values_to_colormap(values, cmap='viridis'):
    norm = plt.Normalize(vmin=np.min(values), vmax=np.max(values))
    colormap = plt.get_cmap(cmap)
    colors = [colormap(norm(value))[:3] for value in values]
    return colors

In [ ]:
fig = PaperFigure(figsize=(7, 5))

fig.set_tick_length(1)
fig.set_font_size(6)
fig.add_background()

# Adding axes ----------------------------------------------------------------------------

ratio1 = (470 - 90) / (500 - 250)
w = 2
fig.add_axes('tectum', (0, 0), ratio1 * w, w)

N = 6
ratio2 = (470 - 280) / (500 - 250)
w = 2 / 3
x = 3.5
pad = ((7 - x) - N * (ratio2 * w)) / (N - 1)

for i in range(N):
    fig.add_axes(f'neuron{i}', (x + i * (ratio2 * w + pad), 0), ratio2 * w, w)
    fig.add_axes(f'neuron{i+N}', (x + i * (ratio2 * w + pad), w), ratio2 * w, w)
    fig.add_axes(f'neuron{i+int(2 * N)}', (x + i * (ratio2 * w + pad), 2*w), ratio2 * w, w)

w = 1.25
pad = (7 - 5 * w) / 4
fig.add_axes('cutoff', (0, 2.5), w, w)
fig.add_axes('histogram1', (2 * (w + pad), 2.5), w, w)
fig.add_axes('histogram2', (4 * (w + pad), 2.5), w, w)


fig.set_line_thickness(0.5)

# Filling axes ---------------------------------------------------------------------------

ax = fig.axes['tectum']
colors = values_to_colormap(somas[neurons_in_tectum, 1], cmap='rainbow')
ax.imshow(atlas.XYprojection, cmap='binary', alpha=0.25, aspect='auto')
for j, i in enumerate(neurons_in_tectum):
    neurons.plot_top(ax, i, color=colors[j], alpha=0.75, soma_size=2, linewidth=0.25, terminals=True, terminal_size=0.25, rasterized=True)
ax.set_xlim([90, 470])
ax.set_ylim([500, 250])
ax.axis('off')

order = np.argsort(somas[:, 1][selected_ids])
uniform_ids = np.linspace(0, len(selected_ids) - 1, 3 * N, endpoint=True).astype('int')
for i in range(3 * N):
    ax = fig.axes[f'neuron{i}']
    neuron_id = selected_ids[order[uniform_ids[i]]]
    ax.imshow(atlas.XYprojection, cmap='binary', alpha=0.25, aspect='auto')
    neurons.plot_top(ax, neuron_id, color='black', alpha=0.75, soma_size=2, linewidth=0.25, terminals=True, terminal_size=0.25, rasterized=False)
    ax.set_xlim([280, 470])
    ax.set_ylim([500, 250])
    ax.axis('off')

ax = fig.axes['cutoff']
x = np.arange(50)
ax.plot(x, diagonal_values, color='black', linewidth=0.75)
ax.plot(x, piecewise_fit, color='red', linewidth=0.75)
ax.set_xticks([0, 49])
ax.spines[['top', 'right']].set_visible(False)

bins = np.linspace(0, 200, 30, endpoint=True)
ax = fig.axes['histogram1']
ax.hist(radii1, bins=bins, color=[0.75, 0.75, 0.75])
ax.fill_betweenx([0, 100], radius_lower, radius_upper, color='red', alpha=0.2, edgecolor='None')
ax.axvline(np.median(radii1), color='blue', linewidth=1)
ax.axvline(predicted_radius, color='red', linewidth=1)
ax.spines[['top', 'right']].set_visible(False)
ax.set_xlim([0, 150])
ax.set_ylim([0, 90])

ax = fig.axes['histogram2']
ax.hist(radii2, bins=bins, color=[0.75, 0.75, 0.75])
ax.fill_betweenx([0, 100], radius_lower, radius_upper, color='red', alpha=0.2, edgecolor='None')
ax.axvline(np.median(radii2), color='blue', linewidth=1)
ax.axvline(predicted_radius, color='red', linewidth=1)
ax.spines[['top', 'right']].set_visible(False)
ax.set_xlim([0, 150])
ax.set_ylim([0, 60])

fig.show()

In [ ]:
fig.save('../Figures/figure7_incomplete.svg')

# Manually annotated values

In [ ]:
predicted_radius

In [ ]:
np.median(radii1) 

In [ ]:
(np.median(radii1) - predicted_radius) / np.median(radii1)

In [ ]:
np.median(radii2)

In [ ]:
(np.median(radii2) - predicted_radius) / np.median(radii1)

In [ ]:
len(neurons_in_tectum)